In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.ndimage import distance_transform_edt
from skimage.morphology import skeletonize, remove_small_objects, binary_closing, disk

# ---------------- CONFIG ----------------
MASK_DIR = r"masks"
OUTPUT_DIR = r"results"
CSV_PATH = os.path.join(OUTPUT_DIR, "stenosis_results.csv")

MIN_MASK_PIXELS = 200
MIN_DIAMETERS = 20
# ----------------------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------- Helper functions --------

def clean_mask(mask):
    mask = mask > 0

    mask = cv2.morphologyEx(
        mask.astype(np.uint8),
        cv2.MORPH_CLOSE,
        np.ones((3,3), np.uint8),
        iterations=1
    )

    mask = remove_small_objects(mask.astype(bool), min_size=50)
    mask = binary_closing(mask, disk(1))

    return (mask * 255).astype(np.uint8)

def keep_largest_component(mask):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

    if num_labels <= 1:
        return mask

    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    out = np.zeros_like(mask)
    out[labels == largest] = 255
    return out

def compute_diameter(mask, skeleton):
    if np.sum(skeleton) == 0:
        return np.array([])

    dist = distance_transform_edt(mask > 0)
    return dist[skeleton] * 2

def compute_stenosis_percent(diameters):
    if len(diameters) < MIN_DIAMETERS:
        return None, None, None

    diameters = diameters[
        (diameters > np.percentile(diameters, 10)) &
        (diameters < np.percentile(diameters, 90))
    ]

    if len(diameters) < 10:
        return None, None, None

    D_min = np.min(diameters)
    D_ref = np.mean(np.sort(diameters)[-10:])

    stenosis = (1 - D_min / D_ref) * 100
    return stenosis, D_min, D_ref

# -------- Batch processing --------

rows = []

mask_files = [f for f in os.listdir(MASK_DIR)
              if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for fname in tqdm(mask_files):
    mask_path = os.path.join(MASK_DIR, fname)
    mask = cv2.imread(mask_path, 0)

    if mask is None:
        continue

    mask = clean_mask(mask)
    mask = keep_largest_component(mask)

    mask_pixels = np.sum(mask > 0)
    if mask_pixels < MIN_MASK_PIXELS:
        continue

    skeleton = skeletonize(mask > 0)
    skeleton_pixels = np.sum(skeleton)

    diameters = compute_diameter(mask, skeleton)
    stenosis, D_min, D_ref = compute_stenosis_percent(diameters)

    if stenosis is None:
        continue

    rows.append({
        "image": fname,
        "mask_pixels": mask_pixels,
        "skeleton_pixels": skeleton_pixels,
        "D_min_px": round(float(D_min), 3),
        "D_ref_px": round(float(D_ref), 3),
        "stenosis_percent": round(float(stenosis), 2)
    })

# -------- Save CSV --------

df = pd.DataFrame(rows)
df.to_csv(CSV_PATH, index=False)

print(f"Saved results to {CSV_PATH}")
print(f"Processed {len(df)} valid cases")


  0%|          | 0/1233 [00:00<?, ?it/s]C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_26676\3527443844.py:32: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later). To avoid this warning, please use the parameter `max_size` instead. For more details, see the documentation of `remove_small_objects`. Note that the new threshold removes objects smaller than **or equal to** its value, while the previous parameter only removed smaller ones.
  mask = remove_small_objects(mask.astype(bool), min_size=50)
C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_26676\3527443844.py:33: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  mask = binary_closing(mask, disk(1))
C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_26676\3527443844.py:32: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later

Saved results to results\stenosis_results.csv
Processed 1141 valid cases
